# Feature Engineering

In [36]:
import pandas as pd 
import numpy as np 
import os
import sys

# Get the absolute path to the 'util' directory (assuming 'eda.ipynb' is one level down from 'your_project')
data_dir = os.path.abspath(os.path.join('data'))

# Add the 'util' directory to sys.path if it's not already there
if data_dir not in sys.path:
    sys.path.append(data_dir)
data_dir

util_dir = os.path.abspath(os.path.join('utils'))

# Add the 'util' directory to sys.path if it's not already there
if util_dir not in sys.path:
    sys.path.append(util_dir)
util_dir

'/Users/shireen/Documents/CaliforniaWildfireAnalysis/utils'

In [39]:
df = pd.read_csv(data_dir+'/supervised_leaned_eda.csv')
df.head()

,ZIP Code,Cov C Amount Weighted Avg,Avg Fire Risk Score,Number of Negligible Fire Risk Exposure,Number of Low Fire Risk Exposure,Number of Moderate Fire Risk Exposure,Number of High Fire Risk Exposure,Number of Very High Fire Risk Exposure,Earned Premium 2020,Earned Exposure 2020,Non-CAT Cov A Smoke - Incurred Losses,Non-CAT Cov A Smoke - Number of Claims,Non-CAT Cov C Fire - Incurred Losses,Non-CAT Cov C Fire - Number of Claims,Non-CAT Cov C Smoke - Incurred Losses,Non-CAT Cov C Smoke - Number of Claims,CAT Cov A Smoke - Incurred Losses,CAT Cov A Smoke - Number of Claims,Earned Premium 2021,Earned Exposure 2021
0,90001,174339.07,0.32,884,407,0,0,0,982193,1291,9320,1,40267,1,5070,1,86803,13,1076066,1345
1,90002,167880.40,0.33,1270,614,0,0,0,1400005,1884,1686,1,20720,1,542,1,27666,5,1523488,1939
2,90003,177789.87,0.31,1195,535,1,0,0,1424103,1731,0,0,128964,2,0,0,49203,6,1537173,1769
3,90004,635509.87,0.45,962,643,56,0,0,3992219,1661,0,0,0,0,0,0,5186,1,4428387,1675
4,90005,852256.91,0.44,224,127,16,0,0,1263229,368,0,0,0,0,0,0,0,0,1377640,379


In [46]:
df["Total CAT Losses"] =  df["CAT Cov A Smoke - Incurred Losses"]
df["Total Non-CAT Losses"] = df["Non-CAT Cov A Fire - Incurred Losses"] + df["Non-CAT Cov A Smoke - Incurred Losses"]
df.rename(columns = { "CAT Cov A Smoke - Number of Claims":"Total CAT Claims"}, inplace= True)
df["Total Non-CAT Claims"] = df["Non-CAT Cov A Smoke - Number of Claims"]+df["Non-CAT Cov C Fire - Number of Claims"]

KeyError: 'Non-CAT Cov A Fire - Incurred Losses'

In [ ]:
df['Avg Premium'] = df['Earned Premium 2020'] / df['Earned Exposure 2020'].replace(0, np.nan)
df['Avg CAT Loss'] = df['Total CAT Losses'] / df['Earned Exposure 2020'].replace(0, np.nan)
df['Avg Non-CAT Loss'] = df['Total Non-CAT Losses'] / df['Earned Exposure 2020'].replace(0, np.nan)
df['Avg CAT Claims'] = df['Total Non-CAT Claims'] / df['Earned Exposure 2020'].replace(0, np.nan)
df['Avg Non-CAT Claims'] = df['Total CAT Claims'] / df['Earned Exposure 2020'].replace(0, np.nan)
df['Avg Premium 2021'] = df['Earned Premium 2021'] / df['Earned Exposure 2021'].replace(0, np.nan)
df['Loss Ratio'] = (df['Total CAT Losses'] + df['Total Non-CAT Losses'])/df['Earned Premium 2020'].replace(0, np.nan)
df['Claim Frequency'] = (df['Total Non-CAT Claims']+df['Total CAT Claims'])/df['Earned Exposure 2020'].replace(0, np.nan)
df['Average Claim Severity'] = (df['Total CAT Losses'] + df['Total Non-CAT Losses'])/(df['Total Non-CAT Claims']+df['Total CAT Claims']).replace(0, np.nan)
# Replace inf with NaN, then drop all NaN values
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)